In [3]:
#%run "../Notebooks/Visualization_functions.ipynb"

loaded variables: 
myparams, myconfiguration_file


In [ ]:
#pip install scikit-spatial
from skspatial.objects import Points, Plane
from skspatial.plotting import plot_3d

import statistics 
from statistics import mode 

In [ ]:
def downsample_big_clouds(list_pcd,
                          labels = None,
                          downsampling = False,
                          downsampling_voxel_size = 0.1,
                          limit_size_point_cloud = 30000, # this is to have point cloud of max a certain size
                          print_statements = False,
                          visualization_on = False
                         ):
    processed_list_pcd = list()
    
    if labels == None:
        labels = [f"Downsampled_pcl_{i}" for i in range(len(list_pcd))]
    
    #print (len(labels),len(list_pcd))
    assert len(labels) == len(list_pcd)
    
    for i,pcd in enumerate(list_pcd):
        if print_statements == True:
            print (f"\n{labels[i]}")
        
        #initialization
        thisdownsampling_voxel_size =  copy.deepcopy(downsampling_voxel_size)
        temp_pcd = copy.deepcopy(pcd)
        downsampled_pcd = copy.deepcopy(pcd)
        points_pcd = get_num_points([temp_pcd])[0]
        
        while points_pcd > limit_size_point_cloud:
            downsampling = True
            list_downsampled_pcd = downsample_list_stitches([temp_pcd],
                                                            [labels[i]],
                                                            voxel_size = thisdownsampling_voxel_size,
                                                            visualization_on = visualization_on,
                                                            print_statements = False)
            downsampled_pcd = list_downsampled_pcd[0]
            down_points_pcd = get_num_points([downsampled_pcd])[0]
            thisdownsampling_voxel_size = round(thisdownsampling_voxel_size,2)
            if print_statements == True:
                print(f"Downsampling {labels[i]} from {points_pcd} points with voxel size {thisdownsampling_voxel_size} ")
                
            points_pcd = down_points_pcd
            #increase voxel size gradually
            thisdownsampling_voxel_size = thisdownsampling_voxel_size +0.05
            
        processed_list_pcd.append(downsampled_pcd)
        if print_statements == True:
            print(f"Processed {labels[i]} has {points_pcd} points ")
        
    return processed_list_pcd,downsampling

In [ ]:
def downsample_list_stitches(list_stitches,
                             labels_stitches=None,color_stitches = None ,
                             voxel_size=0.1,
                             take_screen_shot = False,
                             visualization_on = False,
                             print_statements = False):
    """
    The higher the voxel size the less points we have
    """
    l = len(list_stitches)
    #input check
    
    if not isinstance(list_stitches, list): # if the input is a single point cloud
        list_stitches = list(list_stitches)
        
    if labels_stitches is None or len(labels_stitches)< l:
        labels_stitches = ["Downsampled_pcl_"+str(i+1) for i in list(range(l))] # default labels
    else:
        labels_stitches = ["Downsampled_" + labels_stitches[i] for i in list(range(l))]
        
    if color_stitches is None or len(color_stitches)< l:
        color_stitches = [[0,0.5,0.1]]*l # dafualt colors

    
    downsampled_stitches = []
    for i in list(range(l)):
        mytitle = labels_stitches[i]
        this_pcd = list_stitches[i]
        if not isinstance(voxel_size,list):
            down_pcd = this_pcd.voxel_down_sample(voxel_size=voxel_size)
        else:
            down_pcd = this_pcd.voxel_down_sample(voxel_size=voxel_size[i])
        down_pcd.paint_uniform_color(color_stitches[i])
        downsampled_stitches.append(down_pcd)
        
        
        if visualization_on == True:
            custom_draw_geometry(down_pcd,
                             mytitle = mytitle,
                             params = myparams,  # parameter for camera point view, json file via pressing P
                             configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = take_screen_shot,
                             rotate = True)

        if print_statements == True:
            print (mytitle)
            print ("number of points original : " ,len(np.array(this_pcd.points)))
            print ("number of points with voxel down sample :" ,len(np.array(down_pcd.points)))
            #print ("")
        
    return downsampled_stitches

In [ ]:
def exclude_outliers_list_stitches(labels_stitches,list_stitches,color_stitches,
                                   parameters= None, preprocess_outliers = "stat" ,
                                   take_screen_shot = False,
                                   visualization_on = False, 
                                   rotate = False):
    processed_stitches = []
    
    # process outliers with statistical 
    if preprocess_outliers == "stat":
        for i in list(range(len(labels_stitches))):
            mytitle = labels_stitches[i]
            mytitle = "statistical outlier removal for %s"%(mytitle)
            print (mytitle)

            pcd = list_stitches[i]
            
            
            if parameters == None: 
                my_nb_neighbors=100
                my_std_ratio=0.001
            else:
                my_nb_neighbors = parameters[0]
                my_std_ratio = parameters[1]

            parameters = (my_nb_neighbors,my_std_ratio)
            parameters_labels = ("my_nb_neighbors","my_std_ratio")
            mytuples = list(zip(parameters_labels,parameters))

            processed_pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=my_nb_neighbors,
                                                            std_ratio=my_std_ratio)
            processed_stitches.append(processed_pcd)
            
            if visualization_on == True:     
                custom_draw_geometry_outliers(pcd, ind, 
                                          mytitle = mytitle, mytuples = mytuples,
                                          params = myparams,  # parameter for camera point view, json file via pressing P
                                          configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                                          take_screen_shot = take_screen_shot,
                                          #fov_step  = 15,
                                          rotate = rotate)

    # process outlier with radius
    #elif preprocess_outliers == "radius":
    else:
        for i in list(range(len(labels_stitches))):
            mytitle = labels_stitches[i]
            mytitle = "radius outlier removal for %s"%(mytitle)
            print (mytitle)

            pcd = list_stitches[i]

            if parameters == None:
                nb_points=10
                radius=0.3
            else:
                nb_points = parameters[0]
                radius = parameters[1]

            parameters = (nb_points,radius)
            parameters_labels = ("nb_points","radius")
            mytuples = list(zip(parameters_labels,parameters))

            processed_pcd, ind = pcd.remove_radius_outlier(nb_points=10,
                                                            radius=0.5)
            processed_stitches.append(processed_pcd)
            
            
            if visualization_on == True:
                custom_draw_geometry_outliers(pcd, ind, 
                                              mytitle = mytitle, mytuples = mytuples,
                                              params = myparams,  # parameter for camera point view, json file via pressing P
                                              configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                                              take_screen_shot =take_screen_shot,
                                              #fov_step  = 15,
                                              rotate = rotate)
    

    
    
    
    
    return processed_stitches,ind

## clustering 

In [ ]:
# bring here messy code

In [ ]:
def most_common(List): 
    return(mode(List)) 

In [ ]:
def clustering( original_cloud,
                myeps=0.2, 
                mymin_points=10,
               
               mytitle = "clustering",
               params =None, 
               configuration_file = None, 
               take_screen_shot = False,
               rotate = False,
               
               #statements
                print_statement = True,
                visualization_on = False):
    """
    eps = max Euclidean distance btw 2 points is same cluster (chose small epsilon to avoid long comptime)
    min_points = the minimal number of points necessary to create a cluster

    """
    cloud = copy.deepcopy(original_cloud)
     
    mytuples = list(zip(("myeps","mymin_points"),(myeps,mymin_points)))
    #if mytuples is not None:
    mytitle = create_title(mytitle, mytuples)
    
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(cloud.cluster_dbscan(eps=myeps, min_points=mymin_points, print_progress=print_statement))
        #labels = np.array(cloud.cluster_dbscan(eps=0.2, min_points=10, print_progress=print_statement))

    max_label = labels.max()
    
    ##identify the biggest cluster label
    biggest_cluster_label = most_common(labels)
    biggest_cluster_labels = [label for label in labels if label == biggest_cluster_label]
    ## select indexes of points in the biggest cluster
    index_biggest_cluster_labels = [i for i, e in enumerate(labels) if e == biggest_cluster_label]
    
    ##identify the outliers 
    negative_labels = [label for label in labels if label <0]
    
    ## set colors
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    #colors[labels != biggest_cluster_label] = 0 # in black everything that is not main cluster..
    colors[labels < 0] = 0 # in black the outliers...
      

    ##float64 array of shape (num_points, 3), range [0, 1] , use numpy.asarray() to access data
    cloud.colors = o3d.utility.Vector3dVector(colors[:, :3]) 
    
    
    if visualization_on == True: 
        #o3d.visualization.draw_geometries([cloud])
        
        custom_draw_geometry(cloud, 
                             mytitle = mytitle, mytuples = mytuples,
                             params =myparams, 
                             configuration_file = myconfiguration_file, 
                             take_screen_shot = take_screen_shot,
                             rotate = rotate
                            )

    if print_statement == True:
        print ("")
        print (f"point cloud has {max_label + 1} clusters")
        print (f"there are {len(labels)} points in all clusters")
        print (f"there are {len(negative_labels)} points identified as ouliers ")
        print (f"there are {len(biggest_cluster_labels)} points in the biggest cluster ")
    
    ## select only the biggest cluster
    biggest_cluster_cloud = cloud.select_by_index(index_biggest_cluster_labels)
    
    
    return biggest_cluster_cloud

## plane geometry stuff

In [1]:
def point_to_array(point):
    vectors_basis = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    myarray = transform_coordinates(plane.point, [0, 0, 0], vectors_basis)
    return myarray  

In [14]:
def plane_equation(point,normal, print_statement = True):
    d = 0
    for i in range(3):
        d = d + point[i]*normal[i]
    
    a = normal[0]
    b = normal[1]
    c = normal[2]
    
    if print_statement == True:
        print ("point: %s"%point)
        print ("normal: %s"%normal)
        print (f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {-d:.2f} = 0")
        print ("")
    
    return a,b,c,d

In [13]:
def is_point_in_plane(point = [0,0,0],
                      plane_parameters = [0,0,0,0],
                      neg_margin = 0,
                      pos_margin = 0):

    d_point = sum([point[i]*plane_parameters[i] for i in range(3)])
    
    #print (d)
    #print (d_point)
    
    if d -neg_margin  <= d_point <= d+pos_margin:
        return True
    else:
        return False


In [15]:
def custom_remove_plane(original_cloud,
                        nneg_margin = 0.2,
                        ppos_margin = 0.01,
                        
                        #visualization parameters
                        mytitle = "custom_remove_planes", mytuples = None, 
                        params = None, #camera parameters,json file (P)
                        fov_step  = None, 
                        configuration_file = None, #object properties ,json file (O)
                        rotate = False,
                        
                        #statements
                        print_statements = True,
                        visualization_on = False,
                       ):
    """
    cloud = point cloud
    """
    cloud = copy.deepcopy(original_cloud)
    
    ## 
    mytuples = list(zip(("pos_margin","neg_margin"),(ppos_margin,nneg_margin)))
    
    # initialize empty list
    selected_points = []
      
    # print instructions
    if print_statements == True:
        print("")
        print("1) Please pick at least 3 points using [shift + left click]")
        print("   Press [shift + right click] to undo point picking")
        print("2) After picking points, press 'Q' to close the window")
        
    # we need 3 points
    while len(selected_points) != 3: 
        
        # selected points
        # in theory one can select any number of points, but we want three to find the plane
        selected_points = pick_points(cloud)
    
    # access the coordinates of the points
    # we want only 3 points to find the plane
    coordinates_points= [list(np.asarray(cloud.points)[selected_points[i]]) for i in range(3)]
    
    
    # translate in scikit language the newly found coordinates
    points = Points(coordinates_points)
    #identify the plane through the points
    plane = Plane.best_fit(points)

    # for each point in the point cloud, 
    # if it lies on the plane identified by the manually selected 3 points, it gets excluded
    all_points = np.asarray(cloud.points)
    all_indexes = list(range(len(all_points)))
    all_list_points = [list(e) for i, e in enumerate(all_points[:])]
    inliers = [list(all_points[i]) for i in range(len(all_points)) if is_point_in_plane(all_points[i], 
                                                                                      plane.normal,
                                                                                      neg_margin = nneg_margin,
                                                                                      pos_margin = ppos_margin)]
    
                                                                                            
    if print_statements == True:
        print ("")
        print ("plane point: ",plane.point)
        print ("normal to the plane: ",plane.normal) 
        print ("")
        print ("total number of points: ",len(all_points))                                                                                 
        print ("number excluded points : ",len(inliers))
        print ("with pos_margin= %s ; neg_margin= %s" %(ppos_margin,nneg_margin))
        print ("")
        print ("calculating indexes (might take a while...)")
    
    ## find the indexes of the outliers
    #inliers_idx = [i for i, e in enumerate(all_list_points) if e in inliers] #slower
    inliers_idx = [all_list_points.index(e) for e in inliers] #faster
    
    # include the invert of the outliers indexes
    temp_cloud= copy.deepcopy(cloud)
    cloud_new = temp_cloud.select_by_index(inliers_idx, invert=True)
    
    #visualize eventually
    if visualization_on == True:
        
        continue_statement = "y"
        
        while continue_statement == "y":
            
            mytuples,inliers_idx,update_p = update_parameters(mytuples,all_points,plane.normal,inliers_idx)
            
            if update_p == "y":
                # include the invert of the outliers indexes
                temp_cloud= copy.deepcopy(cloud)
                cloud_new = temp_cloud.select_by_index(inliers_idx, invert=True)
        
            #o3d.visualization.draw_geometries([cloud_new])
            #display_inlier_outlier(cloud, inliers_idx)
            custom_draw_geometry_outliers(cloud_new, inliers_idx, 
                                          mytitle = mytitle, mytuples = mytuples, 
                                          params = params, #camera parameters,json file (P)
                                          fov_step  = None, 
                                          configuration_file = configuration_file, #object properties ,json file (O)
                                          rotate = rotate)


            print('Delete the space in red (y/n): ')
            delete_statement = input()

            print('continue finding planes (y/n): ')
            continue_statement = input()
            
            

        if str(delete_statement) == "y":
            return cloud_new 

        else:
            print ("no points deleted")
            return cloud

In [16]:
def update_parameters(mytuples,all_points, normal,inliers_idx):
    print ("current parameters")
    print (*mytuples, sep= "\n")
        
    print('update parameters (y/n): ')
    update_p = input()
    
    if update_p == "y":
        print('update positive margin [0,1]: ')
        ppos_margin = input()
        
        print('update negative margin [0,1]: ')
        nneg_margin = input()
        
    
        mytuples = list(zip(("pos_margin","neg_margin"),(ppos_margin,nneg_margin)))
        
        all_list_points = [list(e) for i, e in enumerate(all_points[:])]
        inliers = [list(all_points[i]) for i in range(len(all_points)) if is_point_in_plane(all_points[i], 
                                                                                      normal,
                                                                                      neg_margin = nneg_margin,
                                                                                      pos_margin = ppos_margin)]

        ## find the indexes of the outliers
        #inliers_idx = [i for i, e in enumerate(all_list_points) if e in inliers] #slower
        inliers_idx = [all_list_points.index(e) for e in inliers] #faster
    
    return mytuples,inliers_idx,update_p

In [2]:
def statistical_outlier_removal(down_pcd,my_nb_neighbors,my_std_ratio):
    mytitle ="Statistical_outlier_removal"

    parameters = (my_nb_neighbors,my_std_ratio)
    parameters_labels = ("my_nb_neighbors","my_std_ratio")
    mytuples = list(zip(parameters_labels,parameters))

    cloud, ind = down_pcd.remove_statistical_outlier(nb_neighbors=my_nb_neighbors,
                                                    std_ratio=my_std_ratio)
    display_inlier_outlier(down_pcd, ind, 
                           mytitle, mytuples,
                           params =myparams, 
                           configuration_file = myconfiguration_file, 
                           take_screen_shot = False,
                           rotate = False,
                           onewindow = True
                          )


In [ ]:
def crop_save_return(pcd):   
    demo_crop_geometry(pcd)
    
    newest_ply = last_file_containing("model", extension = '.ply')
    print (newest_ply)
    cropped_pcd= o3d.io.read_point_cloud(newest_ply)
    
    newest = last_file_containing("Screen", extension = '.png')
    print (newest)
    Im(filename=newest,width = 600) 

    return cropped_pcd